## Imports

Only considering Reddit and Google Reviews for now.

In [181]:
import pandas as pd

In [182]:
greviews = pd.read_csv('../../output/scraped-ns/all-camps-google-reviews.csv').dropna()['Content'].reset_index().rename(columns = {'Content': 'content'})
greviews

,index,content
0,0,Very Professional People down there. Recommend...
1,1,"Unlike what the other reviews may suggest, peo..."
2,2,The place overall is fine. Good experience. Ju...
3,3,I went there for my NS Checkup today 29 August...
4,4,Don't bother showing up at the timing assigned...
...,...,...
73,162,Clear RT or IPPT or IPPT here.
74,163,The location is too ulu. Fifty years ago this ...
75,164,IPPT here every year~
76,165,whete you get rewarded for exercising


In [183]:
reddit = pd.read_csv('../../output/scraped-ns/nationalservicesg_combineddata.csv')['content'].reset_index()
reddit

,index,content
0,0,Thigh muscle pains during NS.
1,1,SCS questions.
2,2,MP vocation. Heard that there is a new MP comm...
3,3,Research on the State of Sport in Singapore (y...
4,4,"New to NS, Down PES?"
...,...,...
689,689,Got memo from the hospital doc mostly just sta...
690,690,what the MO did there is understandable if tha...
691,691,I hope spf mo is same as saf mo regarding saf...
692,692,Elaborate pls. I don't even know what the vide...


In [184]:
unclean = greviews.append(reddit, ignore_index = True).filter(items = ['content'])
corpus = unclean[['content']].replace('\r\n', ' ', regex = True)
corpus['content'] = corpus['content'].str.strip()
corpus
#corpus[corpus.content != '']

,content
0,Very Professional People down there. Recommend...
1,"Unlike what the other reviews may suggest, peo..."
2,The place overall is fine. Good experience. Ju...
3,I went there for my NS Checkup today 29 August...
4,Don't bother showing up at the timing assigned...
...,...
767,Got memo from the hospital doc mostly just sta...
768,what the MO did there is understandable if tha...
769,I hope spf mo is same as saf mo regarding saf...
770,Elaborate pls. I don't even know what the vide...


In [185]:
corpus.to_csv('../../output/scraped-ns/greviewsandredditcorpus.txt', index=False)
print(corpus.shape)

(772, 1)


In [186]:
ls = list(corpus['content'])
corpustxt = ' '.join(map(str, ls))
corpustxt

'Very Professional People down there. Recommended⭐⭐⭐⭐⭐ Very patient kind respectful.. very smooth throughout the Medical Check-up. Unlike what the other reviews may suggest, people here were adequately friendly(like anywhere else in sg). The place is really clean and efficient The place overall is fine. Good experience. Just a tip if any of u going for medical checkup. DONT GO LATE OR ELSE U have to come back other day to complete the rest. I went there for my NS Checkup today 29 August 8am... Here is what I went through:  Gate Entrance: Security Check & Counter, Take a Sticker Pass and just WALK THROUGH the gate... Don\'t be like me...I stood there thinking I can scan the … Don\'t bother showing up at the timing assigned to you, you\'ll end waiting for more than 2 hours only to realise that you\'re the last person in line. Doctors don\'t really care which is fair because they don\'t really have a choice but to be there. This place is a complete waste of space and time. just here to re

In [11]:
#with open('../../output/scraped-ns/greviewsandredditcorpus.txt', 'r', encoding=' utf-8') as file:
#    corpustxt = file.read()
    
#corpustxt

In [7]:
import en_core_web_sm

nlp = en_core_web_sm.load()
print("Pipeline:", nlp.pipe_names)

Pipeline: ['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']


## Statistical Modelling

In [8]:
content = nlp(corpustxt)
len(content.ents)

935

In [9]:
from collections import Counter

labels = [x.label_ for x in content.ents]
Counter(labels)

Counter({'DATE': 227,
         'TIME': 32,
         'ORG': 367,
         'PERSON': 58,
         'CARDINAL': 121,
         'PERCENT': 10,
         'MONEY': 9,
         'FAC': 3,
         'QUANTITY': 9,
         'GPE': 35,
         'ORDINAL': 34,
         'NORP': 15,
         'EVENT': 2,
         'LOC': 7,
         'PRODUCT': 4,
         'WORK_OF_ART': 1,
         'LAW': 1})

In [10]:
from spacy import displacy

displacy.render(nlp(corpustxt), jupyter=True, style='ent')

## Purely Rule-based Matching

Not sure if we want to enhance the statistical model yet (i.e. spaCy's inbuilt named entity recognizer)

In [11]:
from spacy.lang.en import English

nlp = English()
ruler = nlp.add_pipe("entity_ruler").from_disk("patterns.jsonl")

In [12]:
doc = nlp("4 weeks bmt, 5 weeks vocational training. Just relax. BMT will probably be the best part of your NS journey as a Pes C. After you're in your unit you'll miss kranji camp and the 4 hours per day you spend ironing bed and talking cock.")
print([(ent.text, ent.label_) for ent in doc.ents])
    
html = displacy.render(doc, style="ent", page=True, options={"ents": ["TRAINING", "MEDICAL", "LOCATION", "PORTAL", "SERVICE"]})

ValueError: [E155] The pipeline needs to include a morphologizer in order to use Matcher or PhraseMatcher with the attribute POS. Try using `nlp()` instead of `nlp.make_doc()` or `list(nlp.pipe())` instead of `list(nlp.tokenizer.pipe())`.

In [137]:
# doesn't work
cls = spacy.util.get_lang_class("en")
nlp = cls()
pipeline = ["tok2vec", "tagger", "parser", "entity_ruler", "ner", "lemmatizer"]

for component in pipeline:
    nlp.add_pipe(component)

In [191]:
import spacy
nlp = spacy.load("en_core_web_sm")
# pipeline: ["tok2vec", "tagger", "parser", "ner", "attribute_ruler", "lemmatizer"]

In [26]:
ex = "However swimming will be in Hendai camp instead of Khatib Camp when you visit in June."
html = displacy.render(nlp(ex), style="ent", page=True)

In [28]:
ex = "However swimming will be in Hendai camp instead of Khatib Camp and in RT when you visit in June."
html = displacy.render(nlp(ex), style="ent", page=True)

In [29]:
print("Pipeline:", nlp.pipe_names)

Pipeline: ['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer', 'entity_ruler']


In [30]:
displacy.render(nlp(ex), style='dep', jupyter = True, options = {'distance': 120})

In [31]:
colors = {"TRAINING": "#EEE2DF", "BMT": "#ADA8B6", "ICT": "#DCABDF", "RT/IPT": "#D0C4DF", "MEDICAL": "#EE8434", "CAMP": "#CBEFB6", "FCC": "#DDDFDF", "CMPB": "#717ec3", "PORTAL": "#635d5c", "SERVICE": "#9b1d20", "DATE": "#fbba72"}
options = {"ents": ["TRAINING", "BMT", "ICT", "RT/IPT", "MEDICAL", "CAMP", "FCC", "CMPB", "PORTAL", "SERVICE", "DATE"], "colors": colors}
html = displacy.render(nlp(ex), style="ent", page=True, options=options)

In [32]:
print([(ent.text, ent.label_) for ent in nlp(ex).ents])

[('Hendai camp', 'CAMP'), ('Khatib Camp', 'CAMP'), ('RT', 'RT/IPT'), ('June', 'DATE')]


In [225]:
ex2 = "Does that mean that I won't be throwing grenades in BMT?"
displacy.render(nlp(ex2), style='dep', jupyter=True, options = {'distance': 120})
#options = {"compact": True, "bg": "#09a3d5",
#           "color": "white", "font": "Source Sans Pro"}
#displacy.serve(nlp(ex2), style='dep', options = options)


In [34]:
spacy.explain("PROPN")

'proper noun'

In [226]:
import re
import spacy
from spacy.tokenizer import Tokenizer

#infix_re = re.compile(r'''[~]''') # so that tokenizer doesn't split the word by intra hyphen

#def custom_tokenizer(nlp):
#    return Tokenizer(nlp.vocab, infix_finditer=infix_re.finditer)

nlp = spacy.load("en_core_web_sm")
#nlp.tokenizer = custom_tokenizer(nlp)
ruler = nlp.add_pipe("entity_ruler", config={"overwrite_ents": True}).from_disk("patterns.jsonl")

In [ ]:
colors = {"TRAINING": "#EEE2DF", "BMT": "#ADA8B6", "ICT": "#DCABDF", "IPPT": "#bfe1d9", "RT/IPT": "#D0C4DF", "MEDICAL": "#EE8434", "CAMP": "#CBEFB6", "FCC": "#DDDFDF", "CMPB": "#717ec3", "PORTAL": "#635d5c", "SERVICE": "#9b1d20", "LOCATION": "#fbba72"}
options = {"ents": ["TRAINING", "BMT", "ICT", "IPPT", "RT/IPT", "MEDICAL", "CAMP", "FCC", "CMPB", "PORTAL", "SERVICE", "LOCATION"], "colors": colors}
html = displacy.serve(nlp(corpustxt), style="ent", page=True, options=options)


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



In [171]:
doc = nlp(corpustxt)
#print([(ent.text, ent.label_) for ent in doc.ents])

In [224]:
ex = "Does that mean that I won't be throwing grenades in BMT?"
colors = {"TRAINING": "#EEE2DF", "BMT": "#ADA8B6", "ICT": "#DCABDF", "RT/IPT": "#D0C4DF", "MEDICAL": "#EE8434", "CAMP": "#CBEFB6", "FCC": "#DDDFDF", "CMPB": "#717ec3", "PORTAL": "#635d5c", "SERVICE": "#9b1d20", "DATE": "#fbba72"}
options = {"ents": ["TRAINING", "BMT", "ICT", "RT/IPT", "MEDICAL", "CAMP", "FCC", "CMPB", "PORTAL", "SERVICE", "DATE"], "colors": colors}
html = displacy.render(nlp(ex), style="ent", page=True, options=options)

c:\users\tzemin\appdata\local\programs\python\python37\lib\site-packages\spacy\displacy\__init__.py:189: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


In [4]:
entities = [(ent.text, ent.label_) for ent in nlp(ex).ents]
entities

NameError: name 'nlp' is not defined

In [40]:
dataf = pd.DataFrame(entities, columns =['index', 'entity'])
dataf.to_csv("../../output/feature-entity-table.csv", index=False)

In [92]:
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

pattern = [{"TEXT": {"REGEX": "deff?in[ia]tely"}}]
matcher.add("definitely", [pattern])

doc = nlp("definitely, deffinitely, definately")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

5421073421983476317 definitely 0 1 definitely
5421073421983476317 definitely 2 3 deffinitely
5421073421983476317 definitely 4 5 definately


## Sentiment Scoring

In [2]:
import pandas as pd
feats = pd.read_csv("../../output/corpus-refined-features.csv")
#patterns = pd.read_json("patterns.jsonl", lines=True)
#patterns
feats

,Unnamed: 0,index,Avg_sent,Descriptors,Freq
0,0,ippt,0.857268,"here ,took ,take ,take ,take ,needs ,ffi ,appl...",1052
1,2,ipt,0.860323,"take ,perform ,put ,go ,take ,take ,take ,try ...",570
2,3,sessions,0.867581,"screening ,do ,do ,have ,attend ,attend ,atten...",462
3,4,still,0.857143,"it ,you",449
4,5,rt,0.852987,"attend ,take ,do ,do ,u ,complete ,complete ,w...",434
...,...,...,...,...,...
1766,3515,vary,0.857143,"it ,bit",1
1767,3525,mailbox,1.000000,received,1
1768,3538,forsee,0.916667,"they ,issues",1
1769,3544,save,1.000000,you,1


In [3]:
#data = [('medical','MEDICAL'),('checkup','MEDICAL')] #replace this
dataf = pd.DataFrame(entities, columns =['index', 'entity'])
#change all feature to lowercase
dataf['index'] = dataf['index'].str.lower()
#drop duplicates
dataf = dataf.drop_duplicates()

NameError: name 'entities' is not defined

In [271]:
dataf

,index,entity
0,medical,MEDICAL
2,checkup,MEDICAL
4,today 29 august 8am,DATE
5,more than 2 hours,TIME
6,nsf,ORG
...,...,...
1173,broke,PRODUCT
1174,ns,GPE
1196,elaborate,ORG
1197,east coast singapore,LOC


In [281]:
#testing will be the corpus-refined-feature
df = pd.merge(dataf, feats, on = 'index', how='inner').drop(columns=['Unnamed: 0']) #replace test with corpus-refined-features
df

,index,entity,Avg_sent,Descriptors,Freq
0,medical,MEDICAL,0.950000,"reviewing ,reviewing",68
1,checkup,MEDICAL,0.846154,"did ,mo ,when",9
2,nsf,ORG,0.857143,"need ,cool ,vote ,bracket ,vote ,bracket ,pay ...",36
3,staff,SERVICE,0.877778,"professional ,keep ,keep ,rude ,impatient ,exp...",17
4,screening,MEDICAL,1.000000,undergo,5
...,...,...,...,...,...
93,weeks,DATE,0.875000,"it ,have ,have ,have ,wait ,receiving ,need ,e...",61
94,months,DATE,0.861413,"took ,heard ,heard ,means ,have ,difference ,d...",84
95,leave,PERSON,1.000000,plan,2
96,thursday,DATE,1.000000,daylecture,16


In [269]:
pd.unique(df['entity'])

array(['MEDICAL', 'ORG', 'SERVICE', 'CMPB', 'PERSON', 'BMT', 'IPPT',
       'RT/IPT', 'DATE', 'FCC', 'GPE', 'ORDINAL', 'PORTAL', 'CAMP',
       'CARDINAL', 'LOC', 'TIME', 'TRAINING', 'NORP', 'ICT', 'PRODUCT',
       'EVENT'], dtype=object)

In [277]:
df = df[df['entity'].isin(['MEDICAL', 'SERVICE', 'CMPB', 'BMT', 'IPPT', 'RT/IPT', 'FCC', 'PORTAL', 'CAMP', 'TRAINING'])]

In [ ]:
grouped = df.groupby(['entity'])

In [292]:
df[df['entity']=="MEDICAL"]

,index,entity,Avg_sent,Descriptors,Freq
0,medical,MEDICAL,0.950000,"reviewing ,reviewing",68
1,checkup,MEDICAL,0.846154,"did ,mo ,when",9
4,screening,MEDICAL,1.000000,undergo,5
6,mo,MEDICAL,0.841310,"see ,see ,see ,see ,know ,told ,know ,told ,kn...",49
11,specialist,MEDICAL,0.858369,"see ,confirmed ,confirmed ,see ,seeing ,ask ,s...",33
